In [ ]:
%%html
<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at 'In [3]'.</span>

An Exception was encountered at 'In [3]'.

# Analysis of PSSM results
After running the psiblast on the input fasta-file. The difference between the WT amino acid and substitutions observed during the evolution is computed. 
Again, G and P are removed as potential substitutions from the WT sequence and mutations to G and C as well. 
A cutoff is computed as being significant using the mean of the difference score with 2 standard deviations. 

In [1]:
import pandas as pd
import seaborn as sns

In [2]:
date = "20181204"

In [3]:
df_ = pd.read_csv("ascii_mtx_file",skiprows=2,skip_blank_lines=True,  delim_whitespace=True,header=0,
                  names=["WT",u'A', u'R', u'N', u'D', u'C', u'Q', u'E', u'G', u'H', u'I', u'L', u'K',
       u'M', u'F', u'P', u'S', u'T', u'W', u'Y', u'V', u'A.1', u'R.1', u'N.1',
       u'D.1', u'C.1', u'Q.1', u'E.1', u'G.1', u'H.1', u'I.1', u'L.1', u'K.1',
       u'M.1', u'F.1', u'P.1', u'S.1', u'T.1', u'W.1', u'Y.1', u'V.1',"S1","S2"]) # ,nrows=234)

FileNotFoundError: File b'ascii_mtx_file' does not exist

In [ ]:
df_ = df_.dropna()
df_ = df_[df_["WT"] != '*']

In [ ]:
df_["WT_score"] = 0.0
for index, row in df_.iterrows():
    df_.loc[index,"WT_score"] = df_.loc[index,df_.loc[index,"WT"]]

In [ ]:
df_.head()
cols = [u'WT',u'A',u'R',        u'N',        u'D',
              u'C',        u'Q',        u'E',        u'G',        u'H',
              u'I',        u'L',        u'K',        u'M',        u'F',
              u'P',        u'S',        u'T',        u'W',        u'Y',
              u'V',     u'WT_score']
df_ = df_[cols]
df_.WT_score = df_.WT_score.astype(int)
df_.head()

In [ ]:
# WT, Pos, AA, WT_score, AA_score, Diff_score
dffs = []
for index,row in df_.iterrows():
    for j in range(len(row)):
        if(j == 0):
            continue
        elif(df_.columns[j] == "WT_score"):
            continue
        elif(df_.columns[j] ==  df_.loc[index,"WT"] ):
            continue
        elif( row[j] > df_.loc[index,"WT_score"]):
            val_ = str( abs( df_.loc[index,"WT_score"] - row[j]  ) )
            tmp_ = df_.loc[index,"WT"]+","+str(index)+","+df_.columns[j]+","+str(df_.loc[index,"WT_score"])+","+str(row[j])+","+val_
            dffs.append(tmp_)

In [ ]:
with open(date+"_stabilization_phylogenic.csv",'w') as f:
    f.write("WT,Pos,AA,WT_score,AA_score,Diff_score\n")
    for line in dffs:
        f.write(line+"\n")
df_.to_excel(date+"_PSSM_analysis.xlsx")

In [ ]:
df_pssm_result = pd.read_csv(date+"_stabilization_phylogenic.csv")
df_pssm_result.plot(y="Diff_score",kind='hist')

In [ ]:
# remove all G,P,C -> X
df_pssm_result = df_pssm_result[(df_pssm_result["WT"] != 'G') & (df_pssm_result["WT"] != 'P') & \
                               (df_pssm_result["AA"] != 'G') & (df_pssm_result["AA"] != 'C')]

In [ ]:
# get median of values
sns.boxplot(y="Diff_score",data=df_pssm_result)

In [ ]:
df_pssm_result.info()

In [ ]:
cutoff = df_pssm_result["Diff_score"].mean()+2*df_pssm_result["Diff_score"].std()

In [ ]:
df_pssm_result = df_pssm_result[df_pssm_result["Diff_score"] > cutoff]
df_pssm_result.info()

In [ ]:
print("Unique positions: ",len(df_pssm_result["Pos"].unique()) )

In [ ]:
print("Number of amino acid substitutions: ",len(df_pssm_result.index )

In [ ]:
# write to muts.txt
with open("muts.txt", 'w') as f:
    for i,j,k in zip(df_pssm_result.WT, df_pssm_result.Pos,df_pssm_result.AA):
        f.write(i+str(j)+k+'\n')

In [ ]:
!python /Users/pgreisen/pythonscripts/develop/convert_pssm_to_csv/convert_pssm_to_csv.py ascii_mtx_file